# Lesson 4

This unit is about solving problems where the solution can be expressed as a sequence of steps. The unit starts at [video 229](https://www.youtube.com/watch?v=q6M_pco_5Vo&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=229&pp=iAQB).

## The Pouring Problem

We have two glasses that can contain 9 and 4 oz of water respectively, and we have a faucet and a sink where we can refill or empty either glass. The final goal is to have exactly 6 oz of water in the larger glass. The small one can contain any amount.

There are 6 possible actions:

1. Pouring from glass X to glass Y.
2. Pouring from glass Y to glass X.
3. Filling glass X.
4. Filling glass Y.
5. Emptying glass X.
6. Emptying glass Y.

There are two ways to transfer from X to Y and vice-versa. We can pour from X to Y until

1. Y is full.
2. X is empty. 

### Inventory of Concepts

1. Collection of glasses (we start with two but we can generalize).
2. Total capacity of glass Z (Z $\in$ {X, Y}).
3. Current water level in glass Z.
4. Our goal (6 oz in the large glass).
5. The six pouring actions.
6. A notion of a _solution_. For us, a solution is a sequence of steps from the initial to the goal configuration.

The collection of glasses and their current level provides a complete description of the current state of the world.

### Combinatorial Complexity

For each state of the world there are approximately six possible actions (not all possible at all times). If the solution requires $n$ steps, there are in the order of $n^6$ possible configurations. However we don't know in advance the value of $n$, so the actual complexity of the problem is not known in advance. These are examples of **combinatorial optimization** but we will refer to them as **search** problems.

### Exploring the Space

Our solution is a collection of states, where X = 6 and Y can be anything between 0 and 4 inclusive. At any given step of the exploration, the farthest states we have explored represent the **frontier**. We can then distinguish three sets of states.

1. The goal state(s).
2. The explored states that represent the frontier.
3. The explored states that are not in the frontier.

To find a solution, we keep expanding the frontier until it overlaps one of the solution states. In this kind of problems there are two sitations we want to be able to deal with:

1. When there is no path from the frontier to the solution.
2. If there is a path, we want to be able to find it efficiently, and we want to avoid ending in an infinite loop.

If there is a finite number of states, and if there is a path from the start configuration to the solution, we should be able to find it. We want a strategy, i.e., criteria to decide which states we will explore next.

We define **successors** the set of states you can reach from a given state, and the steps needed to get to those states.

We use `X` and `Y` to indicate the total capacity of the glasses and `x` and `y` to indicate their current level. We represent our paths as lists containing an alternation of states and actions: `[state, action, state, ...]`. The frontier is technically a set of states, but we represent it as a list of _paths_. If no solution is found, we return an empty list (for consistency with the success case).

### Solution

The `successors` function returns a dictionary of `state: action` pairs.

In [2]:
def successors(x, y, X, Y):
    """Return a dict of {state: action} pairs describing what can be reached
    from the (x, y) state and how."""
    assert x <= X and y <= Y  # (x, y) is glass levels; X and Y are glass sizes
    return {
        ((0, y + x) if y + x <= Y else (x - (Y - y), y + (Y - y))): "X -> Y",
        ((x + y, 0) if x + y <= X else (x + (X - x), y - (X - x))): "X <- Y",
        (X, y): "fill X",
        (x, Y): "fill Y",
        (0, y): "empty X",
        (x, 0): "empty Y",
    }

The `pour_problem()` function takes the full capacities of the two containers, the goal, the start configuration, and returns a list of state, action pairs.

We generate the initial path, i.e., a list containing the start state, and we add it to the (empty) frontier. We then pop paths from the left of the frontier, and for each pulled path we look at the last state, and compute its successors. The state we pull out is already in the `explored` set. We then go through the successors, add them to the `explored` set, and check whether they contain the goal. If not, we add them to the frontier.

Note that in line 16 we assign the popped path to `path`, but in line 23 we create a new variable `path2`. It is tempting not to use `path2` and append `path + [action, state]`, but this would be a mistake: we are iterating over the successors of `path[-1]`, each one of which defines a new path. If we did not use `path2` we would be appending `[action, state]` from multiple successors. Each path must be kept separate.

In [3]:
from collections import deque

Fail = []

def pour_problem(X, Y, goal, start=(0, 0)):
    """X and Y are the capacity of glasses; (x, y) is current fill levels
    and represents a state. The goal is a level that can be in either glass.
    Start at start state and follow successors until we reach the goal.
    Keep track of frontier and previously explored; fail when no frontier."""
    if goal in start:
        return [start]  # The trivial path.
    explored = set()  # set of states we have visited
    # A path is a list [state, action, state, ...]. A frontier is a list of paths.
    frontier = deque([[start]])  # ordered list of paths we have blazed
    while frontier:
        path = frontier.popleft()
        # The last element of a path is the last visited state.
        (x, y) = path[-1]  # Last state in the first path of the frontier
        for state, action in successors(x, y, X, Y).items():
            # If the state is in explored do nothing.
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if goal in state:
                    return path2
                else:
                    frontier.append(path2)
    return Fail

In [4]:
pour_problem(9, 4, 6, (9, 4))

[(9, 4),
 'empty Y',
 (9, 0),
 'X -> Y',
 (5, 4),
 'empty Y',
 (5, 0),
 'X -> Y',
 (1, 4),
 'empty Y',
 (1, 0),
 'X -> Y',
 (0, 1),
 'fill X',
 (9, 1),
 'X -> Y',
 (6, 4)]

[Video 235](https://www.youtube.com/watch?v=i46-urAXvPU&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=235) describes the use of doctests, which are a convenient way to test our code. Norvig calls them via `print(doctest.testmod())`. **TODO** review video 235.

## The Bridge Problem

This is the problem where there are four people on one side of the bridge (`here`) and they want to cross. People can only cross one or two at a time. Our inventory contains the following concepts:

1. Person.
2. A collection of people. Two collections, actually:
    - A collection of people on the `here` side.
    - A collection of people on the `there` side.
3. The flashlight.
4. The concepts of states and paths we have seen before.

How about the representaiton? We can represent the four people by the number of minutes it takes them to cross the bridge. For a collection of people we have various choices:

- Tuple.
- List.
- Set.
- Frozenset.

Which ones would be well suited? The representation should be hashable because we are going to use an approach similar to the one for the pour problem, where we were adding states to an `explored` set, and sets must contain *hashable* objects.

Tuples and frozensets are hashable. Sets and lists are not hashable. Note that sets, like lists, contain object references, not objects.

In [5]:
s1 = set([10, 20, 30])
s2 = s1
s1.pop()
print(s2)
del s1, s2

{20, 30}


All four of the above representations would be fine, but only tuples and frozensets are hashable. Norvig opts for a representation of a state as `(here, there, t)` where `here` represents everything on this side, `there` is everything on the other side, and `t` is the total elapsed time. `here` and `there` will be represented as frozensets. We include the flashlight in the frozenset as the string `'light'`. At the beginning,`here` is the frozenset `{1, 2, 5, 10, 'light'}` and `there` is the empty frozenset.

What are the successors of the initial state? At the end of the first step the light will go from `here` to `there`, with at least one person. All combinations of sending 1 or 2 people to the other side are acceptable. There are ${4 \choose 2} = 6$ choices for 2 people and 4 for one person, for a total of 10.

### Exercise: write `successors` for the bridge problem

`successors()` takes a state and returns a dictionary `{state, action, ...}`. A state is a tuple `(here, there, t)`. `here` and `there` are frozensets. `t` is an integer. For example, if the initial state is

```python
here = frozenset({1, 2, 5, 10, 'light'})
there = frozenset({})
```

There are 10 possible actions, corresponding to 1 or 2 people crossing the bridge with the light. The resulting state is one or two people and the flashlight who were `here` moving to `there`. Only the side with the flashlight can take an action, so the first step is determining where the flashlight is. The code below contains my implementations of `successors()`.

In [6]:
import itertools as it
from pprint import pprint

def my_bsuccessors(state):
    """Return a dict of {state: action} pairs. A state is a (here, there, t)
    tuple, where here and there are frozensets of people (indicated by their
    times) and/or the 'light', and t is a number indicating the elapsed time.
    Action is represented by '->' for here to there and '<-' for there to
    here."""
    here, there, t = state

    # If light in here send someone. If light in there someone comes back
    # state includes the total elapsed time.
    # Example: here = frozenset(1, 2, 5, 10, 'light'), there = frozenset()
    # state = (here, there, 0)
    # We may want to consider cases where 1 or 2 people cross the bridge.
    if 'light' in here:
        action = '->'
        fro, to = here, there
    else:
        action = '<-'
        fro, to = there, here

    fro = fro.difference({'light'})
    # Options are sending 1 or 2 people to the other side
    options = it.chain(it.combinations(fro, 1), it.combinations(fro, 2))
    if action == '->':
        return {(fro.difference(x), to.union(x).union({'light'}),
                 t + max(x)): action for x in options}
    else:
        return {(to.union(x).union({'light'}), fro.difference(x),
                 t + max(x)): action for x in options}

initial_state = (frozenset({1, 2, 5, 10, 'light'}), frozenset(), 0)
pprint(my_bsuccessors(initial_state))

{(frozenset({2, 10, 5}), frozenset({1, 'light'}), 1): '->',
 (frozenset({1, 10, 5}), frozenset({2, 'light'}), 2): '->',
 (frozenset({1, 2, 5}), frozenset({10, 'light'}), 10): '->',
 (frozenset({1, 2, 10}), frozenset({'light', 5}), 5): '->',
 (frozenset({5, 10}), frozenset({1, 2, 'light'}), 2): '->',
 (frozenset({2, 5}), frozenset({1, 10, 'light'}), 10): '->',
 (frozenset({2, 10}), frozenset({1, 'light', 5}), 5): '->',
 (frozenset({1, 5}), frozenset({2, 10, 'light'}), 10): '->',
 (frozenset({1, 10}), frozenset({2, 'light', 5}), 5): '->',
 (frozenset({1, 2}), frozenset({10, 'light', 5}), 10): '->'}


Norvig's solution is shown below. He is not too happy with the initial choice of the representation. In particular he thinks that the including the flashlight with the people was not an optimal choice. Maybe it would be better to represent the state as `(here, there, t, light)` where `here` and `there` are frozensets containing people only, and `light` can be a binary/boolean variable indicating whether the flashlight is "here" or "there". Still, the choice wasn't so bad as to require a rewrite. Note a couple of things:

1. Actions are not represented as described in the problem specification. Norvig has added information on who crosses.
2. the `for a in here... for b in here` is pretty nice, as it also covers the case when only one person crosses, without the need for using `itertools.combinations()`.

In [7]:
def bsuccessors(state):
    """Return a dict of {state: action} pairs. A state is a (here, there, t)
    tuple, where here and there are frozensets of people (indicated by their
    times) and/or the 'light', and t is a number indicating the elapsed time.
    Action is represented by '->' for here to there and '<-' for there to
    here."""
    here, there, t = state
    if 'light' in here:
        return dict(((here - frozenset([a, b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a, b)),
                      (a, b, '->'))
                      for a in here if a != 'light'
                      for b in here if b != 'light')
    else:
        return dict(((here | frozenset([a, b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a, b)),
                      (a, b, '<-'))
                      for a in there if a != 'light'
                      for b in there if b != 'light')
            
pprint(bsuccessors(initial_state))

{(frozenset({10, 5}), frozenset({1, 2, 'light'}), 2): (2, 1, '->'),
 (frozenset({2, 10}), frozenset({1, 'light', 5}), 5): (5, 1, '->'),
 (frozenset({2, 5}), frozenset({1, 10, 'light'}), 10): (10, 1, '->'),
 (frozenset({2, 10, 5}), frozenset({1, 'light'}), 1): (1, 1, '->'),
 (frozenset({1, 10}), frozenset({2, 'light', 5}), 5): (5, 2, '->'),
 (frozenset({1, 5}), frozenset({2, 10, 'light'}), 10): (10, 2, '->'),
 (frozenset({1, 10, 5}), frozenset({2, 'light'}), 2): (2, 2, '->'),
 (frozenset({1, 2}), frozenset({10, 'light', 5}), 10): (10, 5, '->'),
 (frozenset({1, 2, 10}), frozenset({'light', 5}), 5): (5, 5, '->'),
 (frozenset({1, 2, 5}), frozenset({10, 'light'}), 10): (10, 10, '->')}


We define a couple of utility functions: `path_states()` and `path_actions` which take a path and return, respectively, a list of states and a list of actions. Remember that a path is a list of the form

`[state, action, state, action, state, ....]`

Therefore we just need to return every second element starting from the first and second positions respectively.

In [8]:
def path_states(path):
    "Return a list of states in this path."
    return path[0::2]

def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]

This is the solution to the bridge problem. The main difference with the pouring problem is in the use of `frontier.sort(key=elapsed_time)`. In the pouring problem, the best solution was the shortest (in terms of length of the path). Here the best solution is the one that takes the shortest time. We always want to have the frontier sorted, so that we take the fastest time first.

**TODO** understand clearly in what way the solution to the pouring problem finds the shortest path. Something to do with BFS, probably.

In [9]:
def bridge_problem(here):
    # If light is not already in `here`, add it, and convert to frozenset.
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed) tuple
    # The frontier starts being the initial state.
    frontier = [[(here, frozenset(), 0)]] # ordered list of paths we have blazed
    # If there's nobody here, we are already done.
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                if not here: # That is, nobody left here
                    return path2
                else:
                    frontier.append(path2)
                    frontier.sort(key=elapsed_time)
    return []

def elapsed_time(path):
    return path[-1][2]

pprint(bridge_problem([1, 2, 5, 10]))

[(frozenset({1, 2, 'light', 5, 10}), frozenset(), 0),
 (2, 1, '->'),
 (frozenset({10, 5}), frozenset({1, 2, 'light'}), 2),
 (1, 1, '<-'),
 (frozenset({1, 10, 'light', 5}), frozenset({2}), 3),
 (5, 1, '->'),
 (frozenset({10}), frozenset({1, 2, 'light', 5}), 8),
 (1, 1, '<-'),
 (frozenset({1, 10, 'light'}), frozenset({2, 5}), 9),
 (10, 1, '->'),
 (frozenset(), frozenset({1, 2, 'light', 5, 10}), 19)]


The program runs and returns an answer, but unfortunately it is the wrong one. `my_bsuccessors()` returns the same answer, so the problem is not in the the successors.

The problem is that when we build the frontier we sort it by the elapsed time. This means that we always pop the path with the lowest time first. If we find a suitable solution, we stop there. However it can happen that at step $k$ we have two paths, $p_1$ and $p_2$ with times $t_1 < t_2$, but in the next step, when we add the time of the new nodes, $t_2 < t_1$.

To avoid getting a sub-optimal solution, we need to look at all solutions, and select the best one, instead of greedily going for the first one. We can imagine three approaches:

1. We exhaust the frontier: even if we find a solution from the frontier, we keep going, we visit every node on the frontier, and give each of them a chance.
2. Once we find a solution, every state on the frontier gets one more step.
3. We check later, i.e., when we find a solution, we don't immediately check whether it is a solution, but rather we go ahead and throw it onto the frontier and we only check whether this is a solution when we pull the next element off from the frontier.

Exhausting the frontier won't work, because it may be infinite (not in this problem, though).
Doing one step won't always work either, as in some cases the best solution may require two steps.
Testing later does work, because now everyone on the frontier is guaranteed to be sorted. When we next pull from the frontier, if the pulled path is a goal, it will be the fastest path.

**QUESTION** my understanding is that if there is a solution in $k$ steps and a better one in $k+1$ steps, our approach would not find it. Is this true? Check.

In [10]:
def bridge_problem(here):
    # If light is not already in `here`, add it, and convert to frozenset.
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (people-here, people-there, time-elapsed) tuple
    # The frontier starts being the initial state.
    frontier = [[(here, frozenset(), 0)]] # ordered list of paths we have blazed
    # If there's nobody here, we are already done.
    if not here:
        return frontier[0]
    while frontier:
        path = frontier.pop(0)
        here1, there1, t1 = state1 = path[-1]
        if not here1 or here1 == set(['light']):
            return path
        for (state, action) in bsuccessors(path[-1]).items():
            if state not in explored:
                here, there, t = state
                explored.add(state)
                path2 = path + [action, state]
                frontier.append(path2)
                frontier.sort(key=elapsed_time)
    return []

def elapsed_time(path):
    return path[-1][2]

pprint(bridge_problem([1, 2, 5, 10]))

[(frozenset({1, 2, 'light', 5, 10}), frozenset(), 0),
 (2, 1, '->'),
 (frozenset({10, 5}), frozenset({1, 2, 'light'}), 2),
 (1, 1, '<-'),
 (frozenset({1, 10, 'light', 5}), frozenset({2}), 3),
 (5, 10, '->'),
 (frozenset({1}), frozenset({2, 10, 'light', 5}), 13),
 (2, 2, '<-'),
 (frozenset({1, 2, 'light'}), frozenset({10, 5}), 15),
 (2, 1, '->'),
 (frozenset(), frozenset({1, 2, 'light', 5, 10}), 17)]


We can try a slightly more complex problem, with 7 people, all with different times.

In [11]:
pprint(bridge_problem([1, 2, 3, 5, 7, 11, 13]))

[(frozenset({1, 2, 3, 'light', 5, 7, 11, 13}), frozenset(), 0),
 (2, 1, '->'),
 (frozenset({3, 5, 7, 11, 13}), frozenset({1, 2, 'light'}), 2),
 (1, 1, '<-'),
 (frozenset({1, 'light', 3, 5, 7, 11, 13}), frozenset({2}), 3),
 (7, 5, '->'),
 (frozenset({11, 1, 3, 13}), frozenset({2, 'light', 5, 7}), 10),
 (2, 2, '<-'),
 (frozenset({1, 2, 3, 'light', 11, 13}), frozenset({5, 7}), 12),
 (2, 1, '->'),
 (frozenset({11, 3, 13}), frozenset({1, 2, 'light', 5, 7}), 14),
 (1, 1, '<-'),
 (frozenset({1, 'light', 3, 11, 13}), frozenset({2, 5, 7}), 15),
 (13, 11, '->'),
 (frozenset({1, 3}), frozenset({2, 'light', 5, 7, 11, 13}), 28),
 (2, 2, '<-'),
 (frozenset({1, 2, 3, 'light'}), frozenset({13, 11, 5, 7}), 30),
 (2, 1, '->'),
 (frozenset({3}), frozenset({1, 2, 'light', 5, 7, 11, 13}), 32),
 (1, 1, '<-'),
 (frozenset({1, 'light', 3}), frozenset({2, 5, 7, 11, 13}), 33),
 (3, 1, '->'),
 (frozenset(), frozenset({1, 2, 'light', 3, 5, 7, 11, 13}), 36)]


### Refactoring paths

[Video 254](https://www.youtube.com/watch?v=NHyG_ywFVzw&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=254). We can have two states with identical `here` and `there` but different `t`. Why can this be a problem? Consider the case where we have only two people, one takes 1 and the other takes 1000. If 1 goes back and forth, we will cycle between the same two states (when 1 and 1000 are both "here", and when 1000 is "here" and 1 is "there"). At each step `t` changes, therefore, these are all seen as different states.

**NOTE** this does not appear to be an issue in our case. Why?

In [12]:
pprint(bridge_problem([1, 100]))

[(frozenset({1, 'light', 100}), frozenset(), 0),
 (100, 1, '->'),
 (frozenset(), frozenset({1, 'light', 100}), 100)]


We should get `t` out of the state tuple, and represent the state just as `(here, there)` storing `t` somewhere else. One possibility is to represent paths as `[state, (action, tot_time), ...]`. Let's write a new version of the `bsuccessors()` function.

In [14]:
def bsuccessors2(state):
    """Return a dict of {state: action} pairs. A state is a (here, there)
    tuple, where here and there are frozensets of people (indicated by their
    times) and/or the light."""
    here, there = state
    if 'light' in here:
        return dict(((here - frozenset([a, b, 'light']),
                      there | frozenset([a, b, 'light'])), 
                    (a, b, '->'))
                    for a in here if a != 'light'
                    for b in here if b != 'light')
    else:
        return dict(((here | frozenset([a, b, 'light']),
                      there - frozenset([a, b, 'light'])),
                     (a, b, '<-'))
                    for a in there if a != 'light'
                    for b in there if b != 'light')

pprint(bsuccessors2((frozenset([1, 2, 5, 10, 'light']), frozenset())))

{(frozenset({10, 5}), frozenset({1, 2, 'light'})): (2, 1, '->'),
 (frozenset({2, 10}), frozenset({1, 'light', 5})): (5, 1, '->'),
 (frozenset({2, 5}), frozenset({1, 10, 'light'})): (10, 1, '->'),
 (frozenset({2, 10, 5}), frozenset({1, 'light'})): (1, 1, '->'),
 (frozenset({1, 10}), frozenset({2, 'light', 5})): (5, 2, '->'),
 (frozenset({1, 5}), frozenset({2, 10, 'light'})): (10, 2, '->'),
 (frozenset({1, 10, 5}), frozenset({2, 'light'})): (2, 2, '->'),
 (frozenset({1, 2}), frozenset({10, 'light', 5})): (10, 5, '->'),
 (frozenset({1, 2, 10}), frozenset({'light', 5})): (5, 5, '->'),
 (frozenset({1, 2, 5}), frozenset({10, 'light'})): (10, 10, '->')}


We have to bring time back into the picture. We may also want to consider more general forms of *cost*. We introduce two utility functions. `path_cost()` assumes that the total cost of a path is already stored in the path and that we just need to extract it, without having to comput anything. `bcost()` computes the cost associated with an action, which in our case is the maximum time.

In [15]:
def path_cost(path):
    """The total cost of a path (which is stored in a tuple with the final
    action."""
    # path = [state, (action, tot_cost), state, ....]
    if len(path) < 3:
        return 0
    else:
        action, total_cost = path[-2]
        return total_cost

def bcost(action):
    "Returns the cost (a number) of an action in the bridge problem."
    # An action is an (a, b, arrow) tuple. a and b are times, arrow is
    # a string.
    a, b, arrow = action
    return max(a, b)

This is Norvig's solution (he does not assign it as an exercise). The tricky part is keepin track of the cost and putting in the right place. For each of the successors we compute the cost of the associated action and update the corresponding path cost.

In [ ]:
def bridge_problem2(here):
    here = frozenset(here) | frozenset(['light'])
    explored = set() # set of states we have visited
    # State will be a (peoplelight_here, peoplelight_there) tuple
    # E.g. ({1, 2, 5, 10, 'light'}, {})
    frontier = [[(here, frozenset())]] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        here1, there1 = state1 = final_state(path)
        if not here1 or (len(here1) == 1 and 'light' in here1):
            return path
        explored.add(state1)
        pcost = path_cost(path)
        for (state, action) in bsuccessors2(state1).items():
            if state not in explored:
                # total cost is the cost of the path `pcost` plus the cost of 
                # `action`
                total_cost = pcost + bcost(action)
                # `state` below is the state we end up with
                path2 = path + [(action, total_cost), state]
                add_to_frontier(frontier, path2)
    return Fail

def final_state(path):
    return path[-1]

The complication we haven't dealt with before: there might be two paths that end up in the same state, and we want to choose the best one. We check if there is already a path in the frontier the final state of which is the same as the final state of the current path. If so, check which one has the lower cost. If the old one does, do nothing, otherwise remove the old path from the frontier and append the new one to it. This is quite different from what we did before, and there isn't much of an explanation for this part.

In [20]:
def add_to_frontier(frontier, path):
    "Add path to frontier, replacing costlier path if there is one."
    # (This could be done more efficiently.)
    # Find if there is an old path to the final stateof this path.
    old = None
    for i, p in enumerate(frontier):
        if final_state(p) == final_state(path):
            old = i
            break
    if old is not None and path_cost(frontier[old]) < path_cost(path):
        return # Old path was better - do nothing
    elif old is not None:
        del frontier[old] # Old path was worse; delete it
    ## Now add the new path and re-sort
    frontier.append(path)

In [19]:
pprint(bridge_problem2([1, 2, 5, 10]))

[(frozenset({1, 2, 'light', 5, 10}), frozenset()),
 ((2, 1, '->'), 2),
 (frozenset({10, 5}), frozenset({1, 2, 'light'})),
 ((2, 2, '<-'), 4),
 (frozenset({10, 2, 'light', 5}), frozenset({1})),
 ((5, 10, '->'), 14),
 (frozenset({2}), frozenset({1, 10, 'light', 5})),
 ((1, 1, '<-'), 15),
 (frozenset({1, 2, 'light'}), frozenset({10, 5})),
 ((2, 1, '->'), 17),
 (frozenset(), frozenset({1, 2, 'light', 5, 10}))]


Overall, this is tricky, and there are many special cases that can pop up. There are a couple of things we can do to avoid mistakes:

1. Write lots of tests. We didn't for speed, but we should have written many more.
2. Use and reuse tools. Every time we don't reuse a tool, we incerase the probability of introducing mistakes.

The function we have written can only solve the bridge problem. We want to write a more general function that can solve a broad range of problems. 

## Missionaries and Cannibals

Let's start considering a shortest path problem (i.e., one where we try to solve the problem with the smallest number of steps) rather than a minimum cost problem, as the latter tend to be tricky. A classic shortest path problem is "missionaries and cannibals" [video 260](https://www.youtube.com/watch?v=stujtjl62NY&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=260). We have 6 people, three of which are missionaries and 3 are cannibals. All 6 people are on one side of a river with a canoe. The goal is to get everybody over to the other side. There are 2 rules:

1. At most 2 people can be in the boat.
2. If we leave more cannibals than missionaries on either side of the river, the cannibals are going to eath the missionaries.

We therefore must shuttle people back and forth in such a way that 2 never occurs. Let's start thinking of possible representations. We could use:

1. One set of missionaries, one set of cannibals and a boolean for the boat, indicating the missionaries, cannibals on this side, and whether the boat is here or there.
2. 3 integers indicating the number of missionaries, cannibals and boats on this side.
3. 6 integers indicating the number of missionaries, cannibals and boats on either side. Let's call these numbers $m_1, m_2, c_1, c_2, b_1, b_2$.

All these representations would work. If we want to genearalize, however, some of those representations may not work. We want to handle initial states with *any* number of missionaries, cannibles, and boats on either side of the river. In this case, since we don't know the initial number, representations that assume that there are, say, $n_m$ missionaries on one side and $N_m - n_m$ on the other, where $N_m$ is the total number of missionaries, would not work. We need all the numbers, i.e., the third representation.

Now we want to write a function that computes the successors for this general case. If the input state is one where the cannibals eat the missionaries, we do not return a successor, i.e., we return the empty dictionary. Norvig introduces also a couple of auxiliary functions that compute the pairwise sum and difference of states and deltas. The function below allows for any number of boats, but at any one step, only one boat can go, i.e., we do not allow multiple boats to cross the river in one step.

In [13]:
def csuccessors(state):
    """Find successors (including ones that result in dining) to this state.
    But a state where the cannibals can dine has no successors."""
    m1, c1, b1, m2, c2, b2 = state
    # N.B.: if there are no missionaries, there's nothing to eat.
    if c1 > m1 > 0 or c2 > m2 > 0:
        return {}
    items = []
    if b1 > 0:
        items += [(sub(state, delta), a + '->')
                  for delta, a in deltas.items()]
    if b2 > 0:
        items += [(add(state, delta), '<-' + a)
                  for delta, a in deltas.items()]
    return dict(items)

deltas = {(2, 0, 1, -2, 0, -1): 'mm',
          (0, 2, 1, 0, -2, -1): 'cc',
          (1, 1, 1, -1, -1, -1): 'mc',
          (1, 0, 1, -1, 0, -1): 'm',
          (0, 1, 1, 0, -1, -1): 'c'}

# IMPORTANT: note the use of zip to operate on pairs of elements from the 
# two tuples.
def add(state, delta):
    return tuple(x + y for x, y in zip(state, delta))

def sub(state, delta):
    return tuple(x - y for x, y in zip(state, delta))

The function seems to be working (we should add proper tests).

In [17]:
csuccessors((3, 3, 1, 0, 0, 0))

{(1, 3, 0, 2, 0, 1): 'mm->',
 (3, 1, 0, 0, 2, 1): 'cc->',
 (2, 2, 0, 1, 1, 1): 'mc->',
 (2, 3, 0, 1, 0, 1): 'm->',
 (3, 2, 0, 0, 1, 1): 'c->'}

The one shown below is Norvig's solution. It does not seem to work, as it returns impossible configurations. **TODO** understand what's wrong.

In [14]:
def mc_problem(start=(3, 3, 1, 0, 0, 0), goal=None):
    """Solve the missionaries and cannibals problem.
    State is 6 ints: (m1, c1, b1, m2, c2, b2) on the start (1) and other (2)
    sides. Find a path that goes from the initial state to the goal state
    (which, if not specified, is the state with no people or boats on the
    start side.)"""
    if goal is None:
        goal = (0, 0, 0) + start[:3]
    if start == goal:
        return [start]
    explored = set() # set of states we have visited
    frontier = [[start]] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in csuccessors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if state == goal:
                    return path2
                else:
                    frontier.append(path2)
    return Fail

In [15]:
mc_problem()

[(3, 3, 1, 0, 0, 0),
 'cc->',
 (3, 1, 0, 0, 2, 1),
 '<-m',
 (4, 1, 1, -1, 2, 0),
 'mc->',
 (3, 0, 0, 0, 3, 1),
 '<-c',
 (3, 1, 1, 0, 2, 0),
 'mm->',
 (1, 1, 0, 2, 2, 1),
 '<-mc',
 (2, 2, 1, 1, 1, 0),
 'mm->',
 (0, 2, 0, 3, 1, 1),
 '<-c',
 (0, 3, 1, 3, 0, 0),
 'cc->',
 (0, 1, 0, 3, 2, 1),
 '<-m',
 (1, 1, 1, 2, 2, 0),
 'mc->',
 (0, 0, 0, 3, 3, 1)]

## Generalization of Shortest-Path Searches

We have the following inventory of concepts from the previous problems. Note that we are doing shortest-path search, not lowest-cost search.

- Paths: `[state, action, state, ...]`
- States: can be atomic. Shortest path search can interface with states through `successors`, `start`, and `goal`.
- Actions: these are also atomic. They don't need to know anything about the representation.
- Successors: a function that takes a state and returns a dictionary of `{state: action,...}`.
- Start: is also atomic.
- Goal: could be a state, or multiple state, possibly many. Rather than using a (possibly large) set, we represent the goal as a function that takes a state and returns a boolean.

We want to write a general `shortest_path_search()` function that returns a path or `Fail` if none is found. What should it take as input? We need to pass the start state, the goal state, and the successors function. We don't need to pass all the states and/or actions because these are returned by `successors()`.

In [19]:
def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set()
    frontier = [[start]]
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return Fail

Let's complete the generalization of the missionaries and cannibals problem. This is my solution.

In [ ]:
def mc_problem2(start=(3, 3, 1, 0, 0, 0), goal=None):
    def is_goal(state):
        return state == (0, 0, 0, 3, 3, 1)
    return shortest_path_search(start, csuccessors, is_goal)

and this is Norvig's solution.

In [22]:
def mc_problem2(start=(3, 3, 1, 0, 0, 0), goal=None):
    if goal is None:
        goal = (0, 0, 0) + start[:3]
    return shortest_path_search(start, csuccessors, all_gone)

def all_gone(state):
    return state[:3] == (0, 0, 0)

In [23]:
mc_problem2()

[(3, 3, 1, 0, 0, 0),
 'cc->',
 (3, 1, 0, 0, 2, 1),
 '<-m',
 (4, 1, 1, -1, 2, 0),
 'mc->',
 (3, 0, 0, 0, 3, 1),
 '<-c',
 (3, 1, 1, 0, 2, 0),
 'mm->',
 (1, 1, 0, 2, 2, 1),
 '<-mc',
 (2, 2, 1, 1, 1, 0),
 'mm->',
 (0, 2, 0, 3, 1, 1),
 '<-c',
 (0, 3, 1, 3, 0, 0),
 'cc->',
 (0, 1, 0, 3, 2, 1),
 '<-m',
 (1, 1, 1, 2, 2, 0),
 'mc->',
 (0, 0, 0, 3, 3, 1)]

## Let's generalize once again

[Video 273](https://www.youtube.com/watch?v=fDgZSxVGtH4&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=273). Let's now generalize the lowest-cost search problems (like the bridge crossing). In addition to the successors, the start, and the goal, we are going to need the cost of each action, `action_cost()`. We will also have the notion of a path cost, but this doesn't need to be passed as an input parameter.

In [ ]:
def lowest_cost_search(start, successors, is_goal, action_cost):
    """Return the lowest cost path, starting from start state,
    and considering successors(state) => {state: action, ...},
    that ends in a state for which is_goal(state) is true,
    where the cost of apath is the sum of action costs,
    which are given by action_cost(action)."""
    explored = set()
    frontier = [[start]]
    while frontier:
        path = frontier.pop(0)
        state1 = final_state(path)
        if is_goal(state1):
            return path
        explored.add(state1)
        pcost = path_cost(path)
        for (state, action) in successors(state1).items():
            if state not in explored:
                total_cost = pcost + action_cost(action)
                path2 = path + [(action, total_cost), state]
                add_to_frontier(frontier, path2)
    return Fail

In [25]:
def bridge_problem2(here):
    start = (frozenset(here) | frozenset(['light']), frozenset())
    return lowest_cost_search(start, bsuccessors2, all_over, bcost)

def all_over(state):
    here, there = state
    return not here or here == set('light')

## Lessons Learned

Some problems require search, i.e., a sequence of steps the number of which we don't know in advance. This is a very large field, and we have only scratched the surface. We have covered two cases: shortest path and least cost search, which are among the most useful. Search is really subtle, and bugs easily creep in. We have two tools to combat them: tests, and standardized tools. Generalization is one of these tools.

## Refactoring the bsuccessors function

[Video 278](https://www.youtube.com/watch?v=CMv0Jhdqup0&list=PLAwxTw4SYaPnJVtPvZZ5zXj_wRBjH0FxX&index=278).